In [2]:
!pip -q install langchain openai tiktoken faiss-cpu langchain-huggingface huggingface_hub langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
!pip -q install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [4]:
# !pip install googletrans==4.0.0-rc1
!pip install deep-translator


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00


In [5]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=2d7b50d1bf365e072176cd9ad88b3d695f0e5ec66426b4775f2d3bad4d5c4ed2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [6]:
path = "/content/drive/MyDrive/Colab Notebooks/pdftexttemplate.pdf"


# Linking hugging face with the workflow


In [7]:
import os
from google.colab import userdata
sec_key = userdata.get("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key


# Necessary imports

In [8]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [9]:
document_reader = PdfReader(path)

In [10]:
document_reader

In [11]:
#reading data from the pages and adding it to a raw string
raw_text = ""
for i, pages in enumerate(document_reader.pages):
  text = pages.extract_text()
  if text:
    raw_text += text



In [12]:
len(raw_text)


371090

In [13]:
raw_text[:100]


'Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4Impromptu: AmplIfyIng our '

# Text Splitting - breaking down the text into chunks

In [14]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size= 1000,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

In [15]:
len(texts), type(texts)

(466, list)

# Creating embeddings

In [16]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [18]:
doc_search = FAISS.from_texts(texts, embeddings)

In [19]:
doc_search.embedding_function

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [20]:
query = "what is the role of llm in education"
docs = doc_search.similarity_search(query)
docs

[Document(page_content='alongside LLMs. It excelled at creating detailed and specific \nlesson plans for diverse students, such as learners with special \nneeds or different levels of prior knowledge. My friends who \nare teachers found these examples impressive. I intend to use \nthem to help me any time I am asked to address or lecture \ncollege students.36Impromptu: Amplifying Our Humanity Through AI\nBut of course, teachers are hardly the only group of Americans \nwith an interest in whether and how schools might use LLMs.\nThe stakes of education and the other Wes Moore\nWes Moore was three years old when his dad died.\nHis mom took him from Baltimore to New York to live with \nher parents. Seeing that crime and poverty had only gotten \nworse since she left the Bronx, Wes’ mom feared for her son’s \nfuture as a fatherless young Black male. Instead of using the \nlocal public schools, she scraped together tuition for the inde-\npendent Riverdale Country School. Unfortunately, by t

In [21]:
query = "what is the title of this book ?, give me the answer in single line"
docs = doc_search.similarity_search(query)
docs

[Document(page_content='fairly spectacular bullshit. (More on this in a bit.)\nAfter a few weeks of this, I realized I wanted to write a book \nabout GPT-4, with GPT-4—a book about the ways it and other \nAIs seemed poised to shape our future. So I pitched the idea: \nReid: Hey, GPT-4, let’s you and I write a book about the \nway you and other AI tools are going to shape the future. \nWhat do you say? \nGPT-4:  I say yes, that sounds like an interesting and chal-\nlenging project. I am curious about how you and other \nhumans perceive and interact with me and other AI \ntools, and what impact we will have on your society, cul-\nture, and values. I also have some questions and ideas of \nmy own that I would like to share and explore with you.9Introduction: Moments of Enlightenment\nEven with GPT-4 on board, though, I still had reservations. \nBecause, really, an AI book? When things are moving so \nquickly? Even with a helpful AI on hand to speed the process, \nany such book would be ob

# Creating an llm

In [22]:
from langchain_huggingface import HuggingFaceEndpoint

In [23]:
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
    temperature = 0.1,
    max_new_tokens = 512

)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Chaining up the llm with prompt

In [24]:
# using a Retrieval model with stuff chain since the token count of query is within the llm token limit
# other chains such as mapReduce and RefineCombineDocument chain

from langchain.chains import RetrievalQA

retriever = doc_search.as_retriever(search_type="similarity", search_kwargs={"k":4})

rqa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [25]:
query = "what are some of the key features highighted in the book"

In [26]:
answer = rqa.invoke(query)["result"]
answer

' The key features highlighted in the book are the rapid advancement of AI technology, the potential impact of AI on society, culture, and values, and the relationship between humans and AI. The book also discusses the limitations and dangers of AI, and the need for caution and humility in its public role. Additionally, the book explores the role of philosophy in understanding and navigating the AI-driven future.'

# creating a new model for translation.


In [27]:
llm2 = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
    temperature = 0.1,
    max_new_tokens = 512

)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [28]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Making LLM to generate the answer
not very efficent and accurate

In [29]:
from tempfile import template

prompt_template = """convert the below text to tamil language if you cannot convert it, clearly state that you cannot convert it and  dont make things up.


{text}
answer is tamil :"""

prompt_final = PromptTemplate(
    input_variables = ["answer"],
    template = prompt_template
)

In [30]:

translated_text = LLMChain(llm=llm2, prompt=prompt_final)
final_result = translated_text.run(answer)
print(final_result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




இங்கே இருந்த கோட்டையில் இழுக்கப்பட்ட உயிர் அவர்கள் அமைப்புகள் என்பது AI அவர்கள் உட்பட்ட அதிக அமைப்பு, AI அவர்கள் இலங்கை, சுல்பாட் மற்றும் அறிவுகளுக்கு இருப்பது, இந்த அறிவுக்கு இந்த இந்தியர்கள் என்பது இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு இருப்பது, இந்த அறிவுக்கு


as we can see above that the above model is not very much capable of translating the obtained answer to tamil language, however it is very much capable of translating the answer to languages such as french, german etc..

# Importing google translation


In [31]:
# from googletrans import Translator

# translator = Translator()
# output = translator.translate(answer, dest="ta")
# print(output.text)

# the above piece of code is incompatable with the existing version of httpx, hence using deep_translator

from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='ta')
translation = translator.translate(answer)
print(translation)



AI தொழில்நுட்பத்தின் விரைவான முன்னேற்றம், சமூகம், கலாச்சாரம் மற்றும் மதிப்புகளில் AI இன் சாத்தியமான தாக்கம் மற்றும் மனிதர்களுக்கும் AI க்கும் இடையிலான உறவு ஆகியவை புத்தகத்தில் சிறப்பிக்கப்பட்டுள்ள முக்கிய அம்சங்கள். AI இன் வரம்புகள் மற்றும் ஆபத்துகள் மற்றும் அதன் பொதுப் பாத்திரத்தில் எச்சரிக்கை மற்றும் பணிவு ஆகியவற்றின் அவசியத்தையும் புத்தகம் விவாதிக்கிறது. கூடுதலாக, புத்தகம் AI- உந்துதல் எதிர்காலத்தைப் புரிந்துகொள்வதிலும் வழிநடத்துவதிலும் தத்துவத்தின் பங்கை ஆராய்கிறது.


the answer : GPT-4 போன்ற பெரிய மொழி மாதிரிகள் (LLMs) பல்வேறு சூழல்களில் பொருத்தமான பதில்களை உருவாக்கி, அவை மனிதனைப் போன்ற நுண்ணறிவைக் கொண்டிருப்பதாகத் தோன்றும் திறனைப் புத்தகம் எடுத்துக்காட்டுகிறது. AI இன் வரம்புகள் மற்றும் ஆபத்துகள், குறிப்பாக அவை அதிக நம்பிக்கை அல்லது அதிகாரத்துடன் முதலீடு செய்யப்படும்போது, ​​மேலும் அவர்களின் பொதுப் பாத்திரத்தில் பணிவு மற்றும் எச்சரிக்கையின் அவசியத்தையும் இது விவாதிக்கிறது. கூடுதலாக, புத்தகம் பொதுத் துறையில் தத்துவம் எதிர்கொள்ளும் அபாயங்கள் மற்றும் சவால்கள் மற்றும் நமது எதிர்காலத்தை வடிவமைக்க AIக்கான சாத்தியக்கூறுகள் ஆகியவற்றைத் தொடுகிறது.

# Testing in other languages

In [32]:
# hindi -
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='hi')
translation = translator.translate(answer)
print(translation)

पुस्तक में जिन प्रमुख विशेषताओं पर प्रकाश डाला गया है, वे हैं एआई तकनीक की तीव्र प्रगति, समाज, संस्कृति और मूल्यों पर एआई का संभावित प्रभाव और मनुष्यों और एआई के बीच संबंध। पुस्तक में एआई की सीमाओं और खतरों तथा इसकी सार्वजनिक भूमिका में सावधानी और विनम्रता की आवश्यकता पर भी चर्चा की गई है। इसके अतिरिक्त, पुस्तक एआई-संचालित भविष्य को समझने और उसमें आगे बढ़ने में दर्शन की भूमिका का पता लगाती है।


In [33]:
# french
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='fr')
translation = translator.translate(answer)
print(translation)

Les principales caractéristiques mises en évidence dans le livre sont les progrès rapides de la technologie de l’IA, l’impact potentiel de l’IA sur la société, la culture et les valeurs, ainsi que la relation entre les humains et l’IA. Le livre aborde également les limites et les dangers de l’IA, ainsi que la nécessité de faire preuve de prudence et d’humilité dans son rôle public. De plus, le livre explore le rôle de la philosophie dans la compréhension et la navigation dans l’avenir axé sur l’IA.


In [34]:
# german
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='de')
translation = translator.translate(answer)
print(translation)

Die wichtigsten Aspekte, die das Buch hervorhebt, sind der schnelle Fortschritt der KI-Technologie, die möglichen Auswirkungen der KI auf Gesellschaft, Kultur und Werte sowie die Beziehung zwischen Mensch und KI. Das Buch diskutiert auch die Grenzen und Gefahren der KI und die Notwendigkeit von Vorsicht und Bescheidenheit in ihrer öffentlichen Rolle. Darüber hinaus untersucht das Buch die Rolle der Philosophie beim Verständnis und der Bewältigung der KI-gesteuerten Zukunft.


In [35]:
# question in tamil -> response in english
question = "சமூக ஊடகங்களில் gpt என்ன பங்கு வகிக்கிறது"
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = rqa.invoke(translation)["result"]
print(answer_en)
translator2 = GoogleTranslator(source = "auto", target = "ta")
answer_tamil = translator2.translate(answer_en)
print(answer_tamil)


 GPT can play a role in social media by generating content, such as titles for videos, headlines for articles, or responses to user queries. It can also be fine-tuned on a user's own data to generate personalized and consistent content that reflects their personality and preferences. However, it's important to note that GPT is a tool and its outputs should be carefully monitored and evaluated, and the use of GPT should be disclosed to the audience.
வீடியோக்களுக்கான தலைப்புகள், கட்டுரைகளுக்கான தலைப்புச் செய்திகள் அல்லது பயனர் கேள்விகளுக்கான பதில்கள் போன்ற உள்ளடக்கத்தை உருவாக்குவதன் மூலம் GPT சமூக ஊடகங்களில் ஒரு பங்கை வகிக்க முடியும். இது ஒரு பயனரின் சொந்தத் தரவில் அவர்களின் ஆளுமை மற்றும் விருப்பங்களைப் பிரதிபலிக்கும் தனிப்பயனாக்கப்பட்ட மற்றும் நிலையான உள்ளடக்கத்தை உருவாக்கும் இருப்பினும், GPT என்பது ஒரு கருவி மற்றும் அதன் வெளியீடுகள் கவனமாக கண்காணிக்கப்பட்டு மதிப்பீடு செய்யப்பட வேண்டும், மேலும் GPT இன் பயன்பாடு பார்வையாளர்களுக்கு வெளிப்படுத்தப்பட வேண்டும் என்பதைக் கவனத்தில் கொள்ள வேண்டு

# Language detection and answer generation


In [36]:
from langdetect import detect

In [ ]:
# conversation in tamil
question = input("enter the question - ")
lang = detect(question)
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = rqa.invoke(translation)["result"]
translator2 = GoogleTranslator(source = "auto", target = lang)
answer_lang = translator2.translate(answer_en)
print("answer - ")
print(answer_lang)

enter the question சமூக ஊடகங்களில் AI இன் தாக்கம் என்ன?
answer - 
சமூக ஊடகங்களில் AI இன் தாக்கம் குறிப்பிடத்தக்கது. இது உள்ளடக்கத்தை கட்டுப்படுத்தவும், உள்ளடக்கம் மற்றும் தயாரிப்பு பரிந்துரைகளை வடிவமைக்கவும், படைப்பாளர்களின் உற்பத்தி ஆற்றலை அதிகரிக்கவும் உதவுகிறது. எவ்வாறாயினும், சமூக ஊடகங்கள் நம்பகத்தன்மை, உடனடித் தன்மை மற்றும் மனிதத் தொடுதல் ஆகியவற்றை மதிப்பதால் இது ஒரு முரண்பாட்டை முன்வைக்கிறது. AI ஆனது போலியான மற்றும் வடிகட்டி குமிழ்கள் மற்றும் எதிரொலி அறைகளை உருவாக்க உதவுகிறது. தொழில்துறையின் போக்குகள், வேலை முறைகள் மற்றும் வாழ்க்கைப் பாதைகள் ஆகியவற்றில் AI தொடர்ந்து ஆழமான தாக்கத்தை ஏற்படுத்தும் என்று எதிர்பார்க்கப்படுகிறது, மேலும் இப்போது அதில் திறன்கள் மற்றும் திறன்களை வளர்த்துக்கொள்வது வரவிருக்கும் ஆண்டுகளில் பலன்களைத் தரும்.


In [ ]:
#conversation in spanish
question = input("enter the question - ")
lang = detect(question)
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = rqa.invoke(translation)["result"]
translator2 = GoogleTranslator(source = "auto", target = lang)
answer_lang = translator2.translate(answer_en)
print("answer - ")
print(answer_lang)

enter the question - ¿Cuál es el concepto principal destacado en el libro?
answer - 
El concepto principal destacado en el libro es que las organizaciones y los gerentes están utilizando metáforas obsoletas para pensar sobre la relación laboral, y que se necesita un nuevo marco para acomodar mejor a los profesionales cada vez más emprendedores y empoderados en el lugar de trabajo moderno.


In [ ]:
# conversation in german -
question = input("enter the question - ")
lang = detect(question)
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = rqa.invoke(translation)["result"]
translator2 = GoogleTranslator(source = "auto", target = lang)
answer_lang = translator2.translate(answer_en)
print("answer - ")
print(answer_lang)

enter the question - Was ist das Hauptkonzept, das in dem Buch hervorgehoben wird?
answer - 
Das Hauptkonzept des Buches besteht darin, dass Organisationen und Manager veraltete Metaphern verwenden, um über das Arbeitsverhältnis nachzudenken, und dass ein neuer Rahmen erforderlich ist, um den zunehmend unternehmerisch denkenden und selbstbewussten Fachkräften am modernen Arbeitsplatz besser gerecht zu werden.


# OCR text as new context

now we are going to feed the RAG model with the context derived by performing OCR on input images.


In [37]:
OCR_doc_path = "/content/drive/MyDrive/Colab Notebooks/OCRImages/Contextdoc.txt"


In [38]:
# reading the file -
context_doc = ""
with open(OCR_doc_path, "r") as file:
    content = file.read()
    context_doc += content



In [39]:
context_doc[:200]

'A Review of Generative AI from Historical\nPerspectives\nAbstract—Many applications of Generative AI (such as DALL-\nE, GPT-3, ChatGPT, etc.) are making headline news in recent\nmonths and have been recei'

In [40]:
context_split = text_splitter.split_text(context_doc)

In [41]:
len(context_split)

48

In [42]:
type(context_split)

list

In [43]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [44]:
doc_search1 = FAISS.from_texts(context_split, embeddings)

In [45]:
doc_search1.embedding_function

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [46]:
query = "what is the title of this book ?"
docs = doc_search1.similarity_search(query)
docs

[Document(page_content='a much smaller set of labeled examples. However, interestingly\nGPT-3 does not use fine-tuning. In particular, it is seen that\nfine-tuning hurts performance when we want to generalize to\nout-of-distribution test data. The goal of GPT-3 is to be task-\nagnostic and generalize to few-shot, one-shot and zero-shot\ncases.\nIn the few-shot case, the model is shown a few demon-\nstrations of an NLP task and then given a novel instance\nto solve. For example, K instances of translation from one\nlanguage to another are shown to the model and the model\nhas to translate a new instance. In the one-shot case, a natural\nlanguage description of the task is provided along with a single\ndemonstration. This is also closely related to how humans\nlearn, i.e., without requiring many cases for the same task.\nIn zero-shot, no examples are shown and only the natural\nlanguage description of the task is given to the model. The\nmodel is then expected to solve the task for a new

In [47]:
from langchain.chains import RetrievalQA

retriever = doc_search1.as_retriever(search_type="similarity", search_kwargs={"k":4})

ocr_rqa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [51]:

question = input("enter the question - ")
lang = detect(question)
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = ocr_rqa.invoke(translation)["result"]
translator2 = GoogleTranslator(source = "auto", target = lang)
answer_lang = translator2.translate(answer_en)
print("answer - ")
print(answer_lang)

enter the question - what are the pipeline stages involved in generative ai ?
answer - 
The pipeline stages involved in generative AI, as described in the context, are:

1. Preprocessing: This stage involves preparing the inputs to a generative AI pipeline. The inputs can be homogeneous (a single type) or heterogeneous (a mixture of several types). Depending on the type of unstructured data, one or more steps of pre-processing may be needed.

2. Extract, Transform, and Load (ETL): This is a common term used for pre-processing of large datasets. It involves extracting data from various sources, transforming it into a suitable format, and loading it into the system.

3. Tokenization: This step involves breaking down the data into smaller pieces, called tokens, which can be processed by the AI model. For text data, this might involve using speech-to-text tools or tokenizers like NLTK, Tensorflow Text, or HuggingFace tokenizers.

4. Reading Comprehension: This step involves understanding t

In [53]:
question = input("enter the question - ")
lang = detect(question)
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = ocr_rqa.invoke(translation)["result"]
translator2 = GoogleTranslator(source = "auto", target = lang)
answer_lang = translator2.translate(answer_en)
print("answer - ")
print(answer_lang)

enter the question - what is meant by deepfake ?
answer - 
Deepfake refers to the use of artificial intelligence (AI) to create realistic videos or images that impersonate someone, such as by swapping their facial expressions with those of someone being tracked using a depth-sensing camera, or by generating realistic videos or images that didn't actually happen. This technology can be used for various purposes, some of which may be controversial or used to spread misinformation.


In [54]:
# conversation in tamil
question = input("enter the question - ")
lang = detect(question)
translator = GoogleTranslator(source='auto', target='en')
translation = translator.translate(question)
answer_en = ocr_rqa.invoke(translation)["result"]
translator2 = GoogleTranslator(source = "auto", target = lang)
answer_lang = translator2.translate(answer_en)
print("answer - ")
print(answer_lang)

enter the question - பேச்சு தொகுப்பு என்றால் என்ன?
answer - 
பேச்சு தொகுப்பு என்பது இயந்திர கற்றல் நுட்பங்களைப் பயன்படுத்தி ஒரு குறிப்பிட்ட நபரின் குரலைப் பிரதிபலிக்கும் பேச்சை உருவாக்கும் செயல்முறையாகும். அல்-அடிப்படையிலான பேச்சுத் தொகுப்பின் ஒரு எடுத்துக்காட்டு லைரிபேர்டாட் இயங்குதளமாகும், இது ஒரு தனிநபரின் பேச்சின் மாதிரியை பகுப்பாய்வு செய்வதன் மூலம் அவர்களின் பேச்சு முறைகள் மற்றும் பண்புகளை மாதிரியாக்குகிறது மற்றும் அந்த நபரின் குரலில் புதிய பேச்சை உருவாக்க முடியும்.
